<a href="https://colab.research.google.com/github/vuchicago/Python/blob/master/DSO_monthly_code_updated10.13.20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


####CODE WRITTEN @VUNGUYEN UPDATED July 22, 2019


####BEFORE YOU INITIALIZE THE CODE, PLEASE GO THROUGH THESE 6 INITIAL STEPS.  


1) Go to google drive to "Shared with me" folder.  Right click on "DSO For SalesForce" folder that has been shared.  
   Select "Add to My Drive" - ***YOU ONLY NEED TO DO THIS ONCE***. IF ALREADY Done IN PREVIOUS MONTHS, THEN SKIP.
   
2)Download all the dso files to inspect.  (**VERY IMPORTANT. WILL ERROR IF DON"T FOLLOW DIRECTIONS)**
       
       a) Make sure that all of the rows and columns are in the SAME EXACT order as the previous month.  
                *For example, Column A or B should be the same fields, even if blank. Sometimes people will delete a column and this will produce a code error
  
        b) Make sure all of the tabs contain data.  If there is a tab in the file that contains no data, then DELETE the tab **

3) Create a folder called "Sales Districts" within the main folder for that month.  
        
        a) Drag all the files that have only only one district or group into this folder.  You can tell by which files contain      "DBAC_REPORT" as a tab      
      
        b) leave all others that contain multiple districts alone (Generally Midwest, North Central, Northeast, Southern, Inside Sales)

4)  Copy the two SG 230 & 490 from the "Sales Districts" folder and move it up to the sales office main folder. Sales Group 230 & 490 are the same as 2000 & 4000, respectively (Melvin Steinmetz 6/7/19 via email).    In the Global tab of the files, change row 39 to either '2000 or '4000 (Add the apostrophe prefix)  .  In cell B55, write "Empty Demarcation"
Put the copies in the main folder & rename the copies as "DBAC DSO Report-Mountain (2000).xls" & "DBAC DSO Report-TXOK (4000).xls", respectively.  

4B) Replace Sales Office with "2000" in the Sales Office row (row 9) in Mountain, and replace Sales office with "4000" in row 9 in TX/OK files

5) Mount Drive using code in the first code block below.  Or Go to Runtime -> Run Selection (Or Click Shift + Enter). Click "Allow" to allow access.  Copy the generated key and enter it where indicated in the popup.


6) Update all of the relevant fields below the mount drive code that says "UPDATE MONTHLY" 



#############  INITIALIZE THE PROGRAM            
FINAL STEP) Run the entire program.  Either Runtime-> Run All.  Or if that doesn't work due to access issues, then run each code block individually by going onto the code block -> Shift + Enter

In [1]:

####MOUNT DRIVE
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)


Mounted at /content/gdrive


In [2]:
import os, shutil, pandas as pd, numpy as np

############################################################# UPDATE MONTHLY   ############################################
year='2020' ###UPDATE ONCE PER YEAR
month_filter=9 ###Update each month up to the end of the previous month (e.g. June if running July)

dso_output_file="dso_sep2020.xlsx" ####UPDATE MONTHLY

dso_main_folder="October (September Data)"  ####UPDATE MONTHLY
dso_prior_month="September (August Data)"

#######            UPDATE MONTHLY
midwest="DBAC  9-30-20  Midwest Districts (1000).xlsx" 
ncentral='DBAC  9-30-20  N. Central Districts (2500).xlsx'
#inside='DBAC  5-31-20  Inside Sales (8500).xlsx' ##Inside is now merged with various other districts
southern='Southern Region DSO September 2020.xlsx'
northeast='DBAC Northeast for Salesforce 9-30-20.xlsx'
txok='Sept 2020 TXOK DSO Comparison.xlsx'
mountain='SEPT 2020 Mountain DSO Comparison.xlsx'


############################################################# DONE WITH UPDATE.  DON'T TOUCH CODE BELOW       ##################################
#########################################################################################################################################

####DIRECTORIES OF FILES

dso_dir='/content/gdrive/My Drive/DSO For Salesforce' ##original dso directory
base_dir=os.path.join(dso_dir,year)
output_dir=os.path.join(base_dir,"DSO_final_output_folder")
main_dir=os.path.join(base_dir,dso_main_folder) 
prior_dir=os.path.join(base_dir,dso_prior_month)

adl_dir=os.path.join(main_dir,"Sales Districts")
#adl_prior_dir=os.path.join(prior_dir,"Sales Districts")

list(os.listdir(main_dir))




['SEPT 2020 Mountain DSO Comparison.xlsx',
 'Sept 2020 TXOK DSO Comparison.xlsx',
 'DBAC  9-30-20  N. Central Districts (2500).xlsx',
 'Southern Region DSO September 2020.xlsx',
 'DBAC  9-30-20  Midwest Districts (1000).xlsx',
 'DBAC Northeast for Salesforce 9-30-20.xlsx']

In [3]:
#print("Number of files in Sales District Subfolder:",len(os.listdir(adl_dir)))
#print("Number of files in Prior Sales District Subfolder:",len(os.listdir(adl_prior_dir)))

In [4]:



#######THE LIST OF DISTRICT MANAGERS
os.chdir(base_dir)                                                               # change directory to base directory
district_managers=pd.read_excel("Cement User ID_SFDC_updateJC_18digit.xlsx",sheet_name="Manager")  ###read the district manager list
district_managers["Sales District Text"]=[str(i) for i in district_managers["Sales District Text"]]
district_managers["USER_C"]=district_managers["User ID- 18 digit"]
col_keep=["Sales District Text","User ID- 18 digit","USER_C"]
district_managers=district_managers[col_keep].rename(columns={"User ID- 18 digit":"OWNERID"})
############################################

targets=pd.read_excel("Targets.xlsx")
targets=targets.rename(columns={"2020 DSO GOALS":"TARGET_DSO_C"})

os.chdir(main_dir)
###Dictionary to map month given to column "PERIOD__C" in the dso upload file
month={"Jan":1,'Feb':2,'Mar':3,'Apr':4,'May':5,"Jun":6,"Jul":7,"Aug":8, "Sep":9,"Oct":10,"Nov":11,"Dec":12}
month_rev = dict([[v,k] for k,v in month.items()]) #reverse dictionary numbered months: 3 letter months

#####Dictionary to map sales office number to "SALES_OFFICE__C" in the dso upload file
sales_office={'1000':"1000 - Midwest",'2000':'2000 - Mountain','4000':'4000 - TXOK','2000, 4000':"2000, 4000 - West",'2500':"2500 - North Central",'3000':"3000 - Southern",
              '7000':"7000 - Northeast",'8500':"8500 - Inside Sales", '9000':"9000 - Trade Sales"}

print("Number of Sales Offices:" ,len(sales_office))
print(targets.columns, "with this many rows:", targets["Level"].shape[0])



Number of Sales Offices: 9
Index(['Level', 'TARGET_DSO_C'], dtype='object') with this many rows: 85


In [5]:
#####CREATE OUTPUT_FN TO READ IN THE TOP ROWS 3-9 in DBAC files sent by regions

def output_top(input_files,i):
    output=pd.DataFrame()
    #####PULLS DATA FROM ROWS 3 TO 9 IN THE DBAC FILES###############
    dbac_info=pd.read_excel(input_files,sheet_name=i,skiprows=1,na_values="NA").iloc[:7] ##top portion of dbac file
    #assert(len(dbac_info.columns.value_counts())>0),print("No Data in tab {} of table {}".format(i,input_files))
    if len(dbac_info.columns.value_counts())==0: ##if worksheet is empty then pass
      pass
      #raise Exception("Exception. No Data in tab {} of table {}".format(i,input_files))
    dbac_info.columns=[i for i,j in enumerate(dbac_info.columns)] ###GRAB and NAME ALL THE COLUMNS
    if dbac_info[1].iloc[0]!="Key Calendar Year":
      raise Exception("Exception.  Column Mismatched (Probably Deleted) in tab {} of table {}".format(i,input_files))
    ###CREATE SEPARATE DATAFRAMES TO GRAB ONLY 4 COLUMNS.  
    temp1=dbac_info[[1,3]] ##COLUMN 1,3,11,13 HAS ALL RELEVANT DATA
    temp2=dbac_info[[11,13]] 
    temp1.columns=[i for i,j in enumerate(temp1)]  ##GIVE THE COLUMNS ARBITARY NAMES
    temp2.columns=[i for i,j in enumerate(temp2)]
    dbac_info=pd.concat([temp1,temp2])
    dbac_info.index=dbac_info[0]
    ####DEFINE SALES GROUP####
    output['filler1']="A"  
    output['filler2']=dbac_info.loc["Key Calendar Year"] 
    output['filler3']="Total (10 + 12)"  
    output['filler4']='N/A'
    if (dbac_info.loc["Sales Group"][1] in ["Empty Demarcation"," "]) & (dbac_info.loc["Sales District"][1] in ["Empty Demarcation"," "]):
        output["REPORT_LEVEL__C"]="Sales Office"
    elif (dbac_info.loc["Sales Office"][1] != "Empty Demarcation") &(dbac_info.loc["Sales District"][1] in ["Empty Demarcation"," "]):
        output["REPORT_LEVEL__C"]="Sales Group"
    else:
        output["REPORT_LEVEL__C"]="Sales District"
    
    #if dbac_info.loc["Sales Office"][1] in ['1000','2000','2500','3000','4000','7000','8500']:
    output["SALES_OFFICE__C"]=sales_office[dbac_info.loc["Sales Office"][1]] ##convert sales office number to num+key using dictionary above
    #else:
     # raise Exception ("Sales Office: {} in TAB: {} within TABLE: {} not an actual SALES OFFICE".format(dbac_info.loc["Sales Office"][1],i,input_files))  ####OUTPUT EXCEPTION EROR IF 
        #####DEFINE SALES DISTRICT     
    if dbac_info.loc["Sales District"][1] in ["Empty Demarcation"," "]:
        output["SALES_DISTRICT__C"]=" "
    else:
        output["SALES_DISTRICT__C"]=dbac_info.loc["Sales District"]
    
    if dbac_info.loc["Sales Group"][1] in ["Empty Demarcation"," "]:
        output["SALES_GROUP__C"]=" "
    else:
        output["SALES_GROUP__C"]=dbac_info.loc["Sales Group"]    

    output['joincol']=1 ###CREATE PRIMARY KEY TO JOIN WITH DBAC_  
    output["YEAR__C"]=dbac_info.loc["Key Calendar Year"]
    output["DIVISION__C"]="Total (10 + 12)" 
    return output.iloc[1:2]


In [6]:
####READS THE BOTTOM HALF OF THE DSO INPUT FILE

def output_bot(input_files,i):
    dbac_bot=pd.read_excel(input_files,sheet_name=i,skiprows=11,index_col=1,na_values="NA").iloc[:12]
    dbac_bot['joincol']=1 ##joincol will act as the primary key
    dbac_bot=dbac_bot[dbac_bot["EOM A/R 1"]!=0]  ###This reads in values in files wheree it's not filler
    dbac_bot["PERIOD__C"]="" ##CREATE PERIOD_C
    for i in range(len(dbac_bot)):
        dbac_bot["PERIOD__C"].iloc[i]=month[dbac_bot.index[i]]
    return dbac_bot

#####MERGES ALL OF THE BOTTOM AND TOP HALF DATAFRAMES CREATED PREVIOUSLY.  IF THERE ARE 
###MULTIPLE TABS, THEN ALL THE ONES GENERATED FROM MULTIPLE TABS ARE MERGED TOGETHER
def dso_salesforce(input_files,tabs):
    output_files=pd.DataFrame()
    
    for i in tabs: 
        ####READ IN FILE AND CREATE MONTH VARIABLE FOR OUTPUT
        dbac_top=output_top(input_files,i) ###USE OUTPUT_FN HERE
        dbac_bot=output_bot(input_files,i)
        dbac_bot=dbac_bot.reset_index()

        output_file=dbac_top.merge(dbac_bot,how='left',on='joincol',suffixes=('_top','_dbac'))  ####MERGE THE BOT AND TOP OF DBAC FILE
        output_file=output_file.rename(columns={"Sales Office":"SALES_OFFICE__C",
                                                "BPDSO 4":"BPDSO__C","DSO 3":"DSO__C",
                                                '12 Month Rolling\nAverage DSO':"ROLLING_DSO__C"})      
    ###CREATE LEVEL VARIABLE IN ORDER TO MATCH IT WITH THE SALES DISTRICT IN THE Cement User ID_SFDC file    
        output_file['Level']=""
        for i in range(output_file.shape[0]):
            if (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales Office"):
                output_file["Level"].iloc[i]=output_file["SALES_OFFICE__C"].iloc[i]
            elif (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales District"):
                output_file["Level"].iloc[i]=output_file["SALES_DISTRICT__C"].iloc[i]
            else:
                output_file["Level"].iloc[i]=output_file["SALES_GROUP__C"].iloc[i]       
        columns=["Level","REPORT_LEVEL__C","SALES_OFFICE__C","SALES_GROUP__C","SALES_DISTRICT__C","PERIOD__C","YEAR__C","DIVISION__C","BPDSO__C","DSO__C","ROLLING_DSO__C"]   
        output_file=output_file[columns]
        
        output_files=output_files.append(output_file,ignore_index=True).dropna()
    return output_files

In [7]:
#####THIS CONSONSOLIDATES FILES THAT ONLY HAVE ONE DISTRICT/GROUP.  THIS READS IT IN THE "Sales Districts" Folder

def dso_salesforce_unconsol(folder,tabs):
    output_files=pd.DataFrame()
    import os
    os.chdir(folder)
    input_files=os.listdir(folder)
    for input_file in input_files:
        for i in tabs:
            
        #input_file=xl.parse(i)  # read a specific sheet to DataFrame    
        ####READ IN FILE AND CREATE MONTH VARIABLE FOR OUTPUT
            dbac_top=output_top(input_file,i) ###USE OUTPUT_FN HERE
            dbac_bot=output_bot(input_file,i)
            dbac_bot=dbac_bot.reset_index()
            
            output_file=dbac_top.merge(dbac_bot,how='left',on='joincol',suffixes=('_top','_dbac'))  ####MERGE THE BOT AND TOP OF DBAC FILE
            output_file=output_file.rename(columns={"Sales Office":"SALES_OFFICE__C",  "BPDSO 4":"BPDSO__C","DSO 3":"DSO__C",
                                                    '12 Month Rolling\nAverage DSO':"ROLLING_DSO__C"})        
            ###CREATE LEVEL VARIABLE IN ORDER TO MATCH IT WITH THE SALES DISTRICT IN THE Cement User ID_SFDC file    
            output_file["Level"]=""
            for i in range(output_file.shape[0]):
                if (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales Office"):
                    output_file["Level"].iloc[i]=output_file["SALES_OFFICE__C"].iloc[i]
                elif (output_file["REPORT_LEVEL__C"].iloc[i]=="Sales District"):
                    output_file["Level"].iloc[i]=output_file["SALES_DISTRICT__C"].iloc[i]
                else:
                    output_file["Level"].iloc[i]=output_file["SALES_GROUP__C"].iloc[i]                  
            ####output the file and append it to the previous output file
            columns=["Level","REPORT_LEVEL__C","SALES_OFFICE__C","SALES_GROUP__C","SALES_DISTRICT__C","PERIOD__C","YEAR__C","DIVISION__C","BPDSO__C","DSO__C","ROLLING_DSO__C"]   
            output_file=output_file[columns]
        
        output_files=output_files.append(output_file,ignore_index=True).dropna()
    return output_files

def west_salesgroup(dset_in):
    for i in range(len(dset_in)):
        if dset_in["SALES_DISTRICT__C"].iloc[i] in ('2314','2317','2319','2315','2230','2219','2212','2213'):
            dset_in["SALES_OFFICE__C"].iloc[i]="2000 - Mountain"
        if dset_in["SALES_DISTRICT__C"].iloc[i] in ('2911','2937', '2936', '2932','2933','2989','2938','2913','2939','2934','2935'):
            dset_in["SALES_OFFICE__C"].iloc[i]="4000 - TXOK"
        else:
            dset_in['SALES_OFFICE__C'].iloc[i]=dset_in["SALES_OFFICE__C"].iloc[i]
        if dset_in["SALES_GROUP__C"].iloc[i]=='230':
            dset_in["SALES_OFFICE__C"].iloc[i]="2000 - Mountain"
        if dset_in["SALES_GROUP__C"].iloc[i]=='490':
            dset_in["SALES_OFFICE__C"].iloc[i]="4000 - TXOK"
        else:
            dset_in['SALES_OFFICE__C'].iloc[i]=dset_in["SALES_OFFICE__C"].iloc[i]            
    return dset_in

In [8]:
#####GRABS ALL THE NAMES OF THE TABS IN THE FILE.  EXCLUDES UNNECESSARY TABS.  Old way.  Too many errors when people add in unncessary tabs
'''

os.chdir(main_dir)
def tab_names(region):
    xl = pd.ExcelFile(region)
    tabs_all=xl.sheet_names  # see all sheet names
    tabs_excl=['Table',
               'DSO_Calc',
               'DaysCostCY',
               'SAPBEXqueries',
               'SAPBEXfilters',
               'DaysCostLY',
               'GrossBalCY',
               'GB_LY',
               'CurRecCY',
               'GB_2Y',
               'Global']
    tabs_incl=[tabs for tabs in tabs_all if tabs not in tabs_excl]
    return tabs_incl

    ###Excludes some non-relevant tabs
midwest_tabs=tab_names(midwest)
ncentral_tabs=tab_names(ncentral)
northeast_tabs=tab_names(northeast)
inside_tabs=tab_names(inside)
southern_tabs=tab_names(southern)
mountain_tabs=['DBAC_REPORT']
txok_tabs=['DBAC_REPORT']
'''

"\n\nos.chdir(main_dir)\ndef tab_names(region):\n    xl = pd.ExcelFile(region)\n    tabs_all=xl.sheet_names  # see all sheet names\n    tabs_excl=['Table',\n               'DSO_Calc',\n               'DaysCostCY',\n               'SAPBEXqueries',\n               'SAPBEXfilters',\n               'DaysCostLY',\n               'GrossBalCY',\n               'GB_LY',\n               'CurRecCY',\n               'GB_2Y',\n               'Global']\n    tabs_incl=[tabs for tabs in tabs_all if tabs not in tabs_excl]\n    return tabs_incl\n\n    ###Excludes some non-relevant tabs\nmidwest_tabs=tab_names(midwest)\nncentral_tabs=tab_names(ncentral)\nnortheast_tabs=tab_names(northeast)\ninside_tabs=tab_names(inside)\nsouthern_tabs=tab_names(southern)\nmountain_tabs=['DBAC_REPORT']\ntxok_tabs=['DBAC_REPORT']\n"

In [17]:
midwest_tabs=['1000', '1189', '1214', '1219', '1222', '1224', '1512', '1513', '1519', '1520', '1521', '1722','1723', '1724', '1728', '1730', '1731', '1732', '1733', '1734', '1739']
ncentral_tabs=['2500', '1113', '1115', '1117', '1118', '1120', '1124', '1125', '1126', '1412', '1413']
northeast_tabs=['SR7000', '203', '228', '229', '231', '233', '236', '238', '261', '262', '264', '265', '266']
southern_tabs=['3000','310','330', '1423', '3121', '3123', '3211', '3212', '3312', '3313','3314', '3315', '3322', '3324', '3433', '3435', '3439'] #'3316' wasn't in Sep 2020
mountain_tabs=['2000','230','2212','2213','2219','2230','2315','2317','2319'] #2219 wasn't in Mar 2020
txok_tabs=['4000','490','2911','2913','2932','2933','2934','2935','2936','2939']##2937 & 2938 wasn't in starting Jan 2020;  not in starting Mar 2020
inside_tabs=['DBAC_REPORT']

mountain_tabs_unconsol=['DBAC_REPORT']
txok_tabs_unconsol=['DBAC_REPORT']


In [10]:
#####call the files 
df_midwest=dso_salesforce(midwest,midwest_tabs)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [11]:
df_midwest

,Level,REPORT_LEVEL__C,SALES_OFFICE__C,SALES_GROUP__C,SALES_DISTRICT__C,PERIOD__C,YEAR__C,DIVISION__C,BPDSO__C,DSO__C,ROLLING_DSO__C
0,1000 - Midwest,Sales Office,1000 - Midwest,,,1,2020,Total (10 + 12),33.810946,31.946585,39.169459
1,1000 - Midwest,Sales Office,1000 - Midwest,,,2,2020,Total (10 + 12),31.620204,30.225819,38.866343
2,1000 - Midwest,Sales Office,1000 - Midwest,,,3,2020,Total (10 + 12),33.527993,32.033324,38.483327
3,1000 - Midwest,Sales Office,1000 - Midwest,,,4,2020,Total (10 + 12),31.767770,31.869821,38.082932
4,1000 - Midwest,Sales Office,1000 - Midwest,,,5,2020,Total (10 + 12),32.307825,32.302104,37.452939
...,...,...,...,...,...,...,...,...,...,...,...
179,1739,Sales District,1000 - Midwest,,1739,5,2020,Total (10 + 12),24.288813,24.670759,29.175376
180,1739,Sales District,1000 - Midwest,,1739,6,2020,Total (10 + 12),23.961604,25.556496,28.294382
181,1739,Sales District,1000 - Midwest,,1739,7,2020,Total (10 + 12),30.508968,29.969615,28.114403
182,1739,Sales District,1000 - Midwest,,1739,8,2020,Total (10 + 12),31.243099,33.260026,28.632328


In [12]:
df_ncentral=dso_salesforce(ncentral,ncentral_tabs)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [13]:
#df_inside=dso_salesforce(inside,inside_tabs) No longer in use

In [16]:
df_northeast=dso_salesforce(northeast,northeast_tabs)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [18]:
df_southern=dso_salesforce(southern,southern_tabs)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [20]:
df_mountain=dso_salesforce(mountain,mountain_tabs)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [23]:
df_txok=dso_salesforce(txok,txok_tabs)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [24]:
df_total=pd.concat([df_midwest,df_ncentral, df_mountain,df_txok,df_southern,df_northeast])
col=["SALES_OFFICE__C","SALES_DISTRICT__C","SALES_GROUP__C"]



In [ ]:
#####call the files 
df_midwest=dso_salesforce(midwest,midwest_tabs)
df_ncentral=dso_salesforce(ncentral,ncentral_tabs)
#df_inside=dso_salesforce(inside,inside_tabs)
df_northeast=dso_salesforce(northeast,northeast_tabs)
df_southern=dso_salesforce(southern,southern_tabs)
df_mountain=dso_salesforce(mountain,mountain_tabs)
df_txok=dso_salesforce(txok,txok_tabs)

df_total=pd.concat([df_midwest,df_ncentral, df_mountain,df_txok,df_southern,df_northeast])
###WEST AND SOUTHERN ARE IN DIFFERENT DIRECTORIES

##################################################
#####Only use this when the tabs are separate workbooks.  FIles are in the "Sales District" folder
#os.chdir(adl_dir)

#west_tabs=['DBAC_REPORT']


#df_west=dso_salesforce_unconsol(adl_dir,west_tabs) 
#df_mountain_txok=west_salesgroup(df_west)  
############################################################





/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
def tab_names(region):
    xl = pd.ExcelFile(region)
    tabs_all=xl.sheet_names  # see all sheet names
    return tabs_all

tab_names(txok)

['4000', '490', '2911', '2913', '2932', '2933', '2934', '2935', '2936', '2939']

In [26]:
df_total

,Level,REPORT_LEVEL__C,SALES_OFFICE__C,SALES_GROUP__C,SALES_DISTRICT__C,PERIOD__C,YEAR__C,DIVISION__C,BPDSO__C,DSO__C,ROLLING_DSO__C
0,1000 - Midwest,Sales Office,1000 - Midwest,,,1,2020,Total (10 + 12),33.810946,31.946585,39.169459
1,1000 - Midwest,Sales Office,1000 - Midwest,,,2,2020,Total (10 + 12),31.620204,30.225819,38.866343
2,1000 - Midwest,Sales Office,1000 - Midwest,,,3,2020,Total (10 + 12),33.527993,32.033324,38.483327
3,1000 - Midwest,Sales Office,1000 - Midwest,,,4,2020,Total (10 + 12),31.767770,31.869821,38.082932
4,1000 - Midwest,Sales Office,1000 - Midwest,,,5,2020,Total (10 + 12),32.307825,32.302104,37.452939
...,...,...,...,...,...,...,...,...,...,...,...
125,0266,Sales District,7000 - Northeast,,0266,6,2020,Total (10 + 12),35.477535,58.889702,60.081377
126,0266,Sales District,7000 - Northeast,,0266,7,2020,Total (10 + 12),40.277653,54.940033,59.514056
127,0266,Sales District,7000 - Northeast,,0266,8,2020,Total (10 + 12),37.427906,56.034127,59.595168
128,0266,Sales District,7000 - Northeast,,0266,9,2020,Total (10 + 12),38.901693,58.769654,59.936887


In [27]:
import numpy as np
df_draft=pd.merge(df_total,district_managers,left_on="Level",right_on="Sales District Text",how='left')
df_draft["Level_final"]=pd.to_numeric(df_draft["Level"],errors='ignore')
###Convert the numbers to float. Keep the strings
df_draft["Level_final"]=df_draft["Level_final"].astype(str).apply(lambda x:str(x.lstrip('0')))






In [28]:
targets["Level"]=(targets["Level"]*1).astype(str)
df_final=pd.merge(df_draft,targets, left_on="Level_final", right_on="Level", how='left')
final_col=["USER_C","OWNERID","REPORT_LEVEL__C","SALES_OFFICE__C","SALES_GROUP__C","SALES_DISTRICT__C","PERIOD__C","YEAR__C","DIVISION__C","BPDSO__C","DSO__C","ROLLING_DSO__C","TARGET_DSO_C","Level_final"]
df_final=df_final[df_final['PERIOD__C']<=month_filter][final_col]
os.chdir(output_dir)    
df_final.to_excel(dso_output_file,index=False)

Documented Problems:

1) One column got deleted in May
2) 7/22/19: Sales District 239 contains no info.  Errored out for NE.  Deleted the tab
3) Added additional rows in the first row for a few tabs.  This caused error  

